Use for previewing and processing raw data 

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Loading food waste database: Joakim_Arvidsson_Food Waste, updated 2023
# Source: https://www.kaggle.com/datasets/joebeachcapital/food-waste

# Load the dataframe
surplus_raw_df = pd.read_csv('raw_data/Joakim_Arvidsson_Food Waste data and research - by country.csv')

# Combine the household and retail food waste columns
surplus_raw_df['Total Food Waste (tons/year)'] = surplus_raw_df['Household estimate (tonnes/year) '] + surplus_raw_df['Retail estimate (tonnes/year) ']

# Filter it so we only have the columns needed
surplus_filtered_df = surplus_raw_df.filter(items=['Country                      ', 'Total Food Waste (tons/year)'])
surplus_filtered_df.rename(columns={'Country                      ': 'Country'}, inplace=True)

surplus_filtered_df.head()

,Country,Total Food Waste (tons/year)
0,Afghanistan,3704135
1,Albania,283550
2,Algeria,4591889
3,Andorra,7485
4,Angola,3667278


### Problem: Even the poorest countries have lots of food waste according to this database
Let's load the food scarcity database

In [3]:
# Loading food scarcity database: Maryam Sikander_Zero_Hunger
# https://www.kaggle.com/datasets/maryamsikander/sdg-2-zero-hunger

# Load the dataframe
scarcity_raw_df = pd.read_excel('raw_data/Maryam Sikander_Zero_Hunger.xlsx', sheet_name="Prevalence-of-food-insecurity")

# Data is provided from years 2015 - 2020. Since our other dataset is recent, we will just take the data from 2020
scarcity_2020_df = scarcity_raw_df[scarcity_raw_df["Year"] == 2020]

# Since year is now always 2020 we can drop the column
scarcity_2020_df = scarcity_2020_df.drop(columns=["Year"])
scarcity_2020_df.head()

,Entity,Code,Prevalence of moderate or severe food insecurity in the total population (%age)
5,Afghanistan,AFG,70.0
11,Africa (FAO),NaN,55.5
17,Albania,ALB,30.9
23,Algeria,DZA,19.0
27,Angola,AGO,77.7


Here we see that our food scarcity database contains percentages of the total population who are experiencing food insecurity. 

It also contains regions in the database such as Northern Europe - these are easily identified by their lack of country code in the "Code" column

We can combine this data with a population database to estimate the number of people experiencing food scarcity per country in 2020: 

In [4]:
# Loading population statistics database: 
# https://www.kaggle.com/datasets/iamsouravbanerjee/world-population-dataset

population_raw_df = pd.read_csv('raw_data/Sourav Banerjee_world_population.csv')
population_filtered_df = population_raw_df.filter(items=["CCA3", "2020 Population"])

population_filtered_df.head()

,CCA3,2020 Population
0,AFG,38972230
1,ALB,2866849
2,DZA,43451666
3,ASM,46189
4,AND,77700


Now, we can use this information to cross-reference countries by their country code and estimate the number of people experiencing food scarcity

In [5]:
# Add the 2020 Population statistics aligned by country code
scarcity_population_df = scarcity_2020_df.join(population_filtered_df.set_index('CCA3'), on='Code')

# Remove any that don't have a Code
scarcity_population_df = scarcity_population_df.dropna(subset=['2020 Population'])

# Create a new column with our estimated number of people experiencing food scarcity
scarcity_population_df['People experiencing scarcity'] = scarcity_population_df['Prevalence of moderate or severe food insecurity in the total population (%age) '] / 100 * scarcity_population_df['2020 Population']

# Round to the nearest person
scarcity_population_df = scarcity_population_df.round(0)

# Filter the just the country and the scarcity number
filtered_scarcity_df = scarcity_population_df.filter(items=["Entity", "People experiencing scarcity"])

filtered_scarcity_df.head()

,Entity,People experiencing scarcity
5,Afghanistan,27280561.0
17,Albania,885856.0
23,Algeria,8255817.0
27,Angola,25973933.0
28,Antigua and Barbuda,30579.0


Finally, we can estimate the tons of food a person eats each year to get comparable numbers for scarcity and surplus

In [6]:
# https://goodseedventures.com/worldwide-food-consumption-per-capita-2/
# ^ The average person consumes around 675 kg of food per year

# Replacing People experiencing scarcity with Tons of food scarcity
filtered_scarcity_df["Tons of food scarcity"] = filtered_scarcity_df['People experiencing scarcity'] * 0.675
filtered_scarcity_df = filtered_scarcity_df.round(0)
filtered_scarcity_df = filtered_scarcity_df.drop(columns=["People experiencing scarcity"])
filtered_scarcity_df.rename(columns={'Entity': 'Country'}, inplace=True)

filtered_scarcity_df.head()

,Country,Tons of food scarcity
5,Afghanistan,18414379.0
17,Albania,597953.0
23,Algeria,5572676.0
27,Angola,17532405.0
28,Antigua and Barbuda,20641.0


We have 214 countries with a food surplus and 146 countries with food scarcity, with lots of overlap of course. 

This indicates that the best option is going to be to try to distribute food that is wasted within that country, instead of importing it from elsewhere. Lets combine the surplus and scarcity dataframes and see

In [7]:
# Strip the country field on this dataframe of any whitespace, it was causing an issue
surplus_filtered_df['Country'] = surplus_filtered_df['Country'].str.strip()

# Join the dataframes on the Country columns, perform an outer join so we don't loose any information
surplus_and_scarcity_df = surplus_filtered_df.join(filtered_scarcity_df.set_index('Country'), on='Country', how='outer')

# Fill NaN with zeroes
surplus_and_scarcity_df.fillna({"Total Food Waste (tons/year)":0, "Tons of food scarcity":0}, inplace=True)

# Reset the index
surplus_and_scarcity_df.reset_index(inplace=True, drop=True)

# Change this option if you'd like to view many rows
pd.set_option('display.max_rows', None)
surplus_and_scarcity_df.head(10)

,Country,Total Food Waste (tons/year),Tons of food scarcity
0,Afghanistan,3704135.0,18414379.0
1,Albania,283550.0,597953.0
2,Algeria,4591889.0,5572676.0
3,Andorra,7485.0,0.0
4,Angola,3667278.0,17532405.0
5,Antigua and Barbuda,8422.0,20641.0
6,Argentina,3943943.0,11247749.0
7,Armenia,321454.0,202635.0
8,Aruba,9220.0,0.0
9,Australia,2801358.0,2061947.0


We should also assume that some large portion of food that is wasted cannot be redistributed due to it spoiling first. 

Let's assume that 50% of the food wasted would not be able to survive redistribution for now. In the future, we could improve this by decreasing the amount of food left available each day depending on how long it takes to transport. 

In [8]:
# Replacing tons of food wasted with tons of food surplus
surplus_and_scarcity_df['Tons of surplus food'] = surplus_and_scarcity_df['Total Food Waste (tons/year)'] * 0.50
surplus_and_scarcity_df.drop(columns=['Total Food Waste (tons/year)'], inplace=True)

surplus_and_scarcity_df.tail(15)

,Country,Tons of food scarcity,Tons of surplus food
212,United Arab Emirates,470169.0,524399.0
213,United Kingdom,1584280.0,2741726.0
214,United Rep. of Tanzania,0.0,3907434.0
215,United States,18594390.0,0.0
216,United States Virgin Islands,0.0,4536.0
217,United States of America,0.0,12255632.0
218,Uruguay,532366.0,150120.5
219,Uzbekistan,5318166.0,1758854.5
220,Vanuatu,49021.0,15993.5
221,Venezuela (Boliv. Rep. of),0.0,1255727.5


Countries like the US might cause some issues since one database had it listed as United States and the other has it as United States of America. 

We'll need location data on these anyway, so that should resolve the issue

Let's try to get location data (GPS coordinates) for each country as well. This will help us pair up countries and we can use the distance to factor in the cost of transport

In [23]:
from geopy.geocoders import Nominatim
import time

# This free API will let us geocode coordinates from the country names
geolocator = Nominatim(user_agent="EE695_Final_Project") 

def geocode_country(country_name):
    try:
        
        location = geolocator.geocode(country_name)
        time.sleep(1) # There is a rate limit of 1 per second
        if location:
            return f"{location.latitude}, {location.longitude}"
        else:
            return None
    except Exception as e:
        print(f"Error geocoding {country_name}: {e}")
        return None



In [24]:
from tqdm import tqdm
tqdm.pandas()
# TQDM for a progress bar. 

### DO NOT run this line of code any more than necessary. The data is saved to a file so that we don't have to clog their API
# surplus_and_scarcity_df['Coordinates'] = surplus_and_scarcity_df['Country'].progress_apply(geocode_country)

### Saving file to CSV and PKL so that we don't need to pull that data again
# surplus_and_scarcity_df.to_pickle("processed_data/Global_Food_Dataset.pkl")
# surplus_and_scarcity_df.to_csv("processed_data/Global_Food_Dataset.csv")

 71%|███████   | 161/227 [03:45<01:29,  1.36s/it]

Error geocoding Portugal: Service timed out


100%|██████████| 227/227 [05:18<00:00,  1.40s/it]


This worked really well, there are few gaps (like Portugal), but these can easily be added by hand. Let's save this to a file so we don't clog their servers again

In [25]:
surplus_and_scarcity_df.head()

,Country,Tons of food scarcity,Tons of surplus food,Coordinates
0,Afghanistan,18414379.0,1852067.5,"33.7680065, 66.2385139"
1,Albania,597953.0,141775.0,"1.2315264, -75.89204331320687"
2,Algeria,5572676.0,2295944.5,"28.0000272, 2.9999825"
3,Andorra,0.0,3742.5,"42.5407167, 1.5732033"
4,Angola,17532405.0,1833639.0,"-11.8775768, 17.5691241"


Let's make sure we can calculate the distance between two countries

In [30]:
import geopy.distance

def calculate_distance(country_1, country_2):
    surplus_and_scarcity_df = pd.read_pickle("processed_data/Global_Food_Dataset.pkl")
    coords_1 = surplus_and_scarcity_df.loc[surplus_and_scarcity_df["Country"] == country_1]["Coordinates"]
    coords_2 = surplus_and_scarcity_df.loc[surplus_and_scarcity_df["Country"] == country_2]["Coordinates"]
    distance_km = geopy.distance.geodesic(coords_1, coords_2).km
    return distance_km

In [33]:
calculate_distance("Argentina", "United States")

# I checked this online and it matches!

9046.120020002556

### Final Changes were made and saved in processed_data/Manually_Checked_Global_Food_Dataset.csv

This will be our final dataset

Used: https://www.gps-coordinates.net/